<a href="https://colab.research.google.com/github/yeop6134/Ai_humanities/blob/main/BERT%EA%B8%B0%EB%B0%98TopicModeling/Korean_CTM_%EB%AA%85%EC%82%AC%EC%82%AC%EC%9A%A9_%EB%B6%88%EC%9A%A9%EC%96%B4%EC%A0%9C%EA%B1%B0_newDiscrimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/인공지능인문학"

/content/drive/MyDrive/Colab Notebooks/인공지능인문학


In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm

In [ ]:
documents = [line.strip() for line in open("newDiscrimination_text.txt", encoding="utf-8").readlines()]

In [ ]:
documents[:5]

['못사는 놈들은 정말 불쌍해.',
 '쟤처럼 느글거리는 인상이 꼭 사고치더라',
 '어린 애들 따 먹어봤다는 거잖아 좀 부럽네',
 '너 계속 이러면 나중에 청소차 몰고 다닌다고 엄마가 말했어 안했어',
 '와 미국인 정서 걍 이해불가던데 어떻게 사귀는거지?']

In [ ]:
korean_stopwords_path = "./korean_stopwords.txt"

# 텍스트 파일을 오픈합니다.
with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.nouns(sent)  # 명사만 토큰화
        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
         if w not in stopwords: 
            result.append(w)

        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
train_bow_embeddings = vectorizer.fit_transform(documents)

In [ ]:
print(train_bow_embeddings.shape)

(4334, 2778)


In [ ]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
len(vocab)

2778

In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

# Combined TM 학습하기

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=8, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [86680/86680]	Train Loss: 20.403340929820892	Time: 0:00:01.338407: : 20it [00:27,  1.36s/it]


# 토픽들
학습 후에는 토픽 모델이 선정한 토픽들을 보려면 아래의 메소드를 사용합니다.  


> get_topic_lists


해당 메소드에는 각 토픽마다 몇 개의 단어를 보고 싶은지에 해당하는 파라미터를 넣어즐 수 있습니다.

In [ ]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['게이',
              '한국',
              '지하철',
              '백인',
              '일본',
              '동성애자',
              '이유',
              '집단',
              '커피',
              '사람'],
             1: ['여자', '남자', '결혼', '얼굴', '여성', '나이', '한남', '매력', '머리', '그게'],
             2: ['서울',
              '한국',
              '나라',
              '사람',
              '조선족',
              '차별',
              '한국인',
              '이해',
              '외국인',
              '백인'],
             3: ['정신',
              '환자',
              '귀머거리',
              '병원',
              '학교',
              '장애',
              '공부',
              '정신병',
              '숙제',
              '특수'],
             4: ['탈출', '빨대', '재미', '벌이', '전환', '촉감', '취업', '화통', '화물차', '주위'],
             5: ['중국', '중국인', '흑인', '냄새', '인종', '거기', '사람', '일본', '세계', '동남아'],
             6: ['장애', '장애인', '가난', '코로나', '의무', '공익', '조상', '수준', '시설', '주제'],
             7: ['후면',
              '파업',
 

- 1번 토픽은 성차별로 토픽화 되었다.
- 2번 토픽은 '한국'에 대한 토픽으로 모였다. (그러나 뒤에 외국인도 있다.)
- 3번 토픽은 '장애'를 토픽으로 볼 수 있을 듯 하다.
- 5번 토픽은 국가, 인종의 토픽인 것 같다.
- 나머지는 잘 모르겠다.
- 위에서 나온 토픽은 다섯가지 이므로 주제를 5가지로 설정해서 모델을 돌려보자.

# 시각화

In [ ]:
%pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)
     

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
Sampling: [10/10]: : 10it [00:14,  1.42s/it]
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=5, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [86680/86680]	Train Loss: 19.290120630479926	Time: 0:00:01.267447: : 20it [00:31,  1.55s/it]


In [ ]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['북한',
              '나라',
              '사람',
              '냄새',
              '가난',
              '조선족',
              '한국인',
              '동양인',
              '난민',
              '얼굴'],
             1: ['여자', '여성', '시집', '차별', '흑인', '화장', '담배', '운전', '페미', '시녀'],
             2: ['남자', '결혼', '여자', '중국', '한국', '게이', '중국인', '얼굴', '군대', '한남'],
             3: ['장애',
              '장애인',
              '그건',
              '귀머거리',
              '정신병',
              '이용',
              '점자',
              '힙합',
              '피부',
              '벙어리'],
             4: ['약속',
              '어딜',
              '운전기사',
              '한잔',
              '인구',
              '주제',
              '정당',
              '과학',
              '조무',
              '맏이']})